### This notebook is not working with the current latest package.
### Let's wait for a new release which is going to include some new features used in this notebook.

# 👨‍💼👨‍🚀👨‍💻 RAG with File Retrieval (image)

This notebook is restricted to OpenAI API only

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.18.0-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;

var kernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: Secrets.OpenAIModel,
        apiKey: Secrets.OpenAIApiKey)
    .Build();

### Let's create a chat group for agents and add a goal to the chat.

In [ ]:
#pragma warning disable SKEXP0110

AgentGroupChat chat = new();

var goal = """
    Give the totals and subtotal from the receipt.
    """;

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, goal));


### Read _details.txt_ file and upload it to the model

In [ ]:
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0001

var fileService = new OpenAIFileService(Secrets.OpenAIApiKey);
var bytes = await System.IO.File.ReadAllBytesAsync("../../Resources/Data/test_image.jpg");
var imageContent = new ImageContent(bytes, mimeType: "image/jpeg"){ InnerContent = "test_image.jpg" };
var executionSettings = new OpenAIFileUploadExecutionSettings("../../Resources/Data/test_image.jpg", OpenAIFilePurpose.Vision);
var uploadFile = await fileService.UploadContentAsync(
    imageContent,
    executionSettings
);

### Create Jack Sparrow agent with file retrieval capabilities (agents are using the out-of-the-box RAG feature of the OpenAI assistants)

In [ ]:
#pragma warning disable SKEXP0110

using System.ClientModel;

var agent = await OpenAIAssistantAgent.CreateAsync(
    kernel: kernel,
    clientProvider: OpenAIClientProvider.ForOpenAI(new ApiKeyCredential(Secrets.OpenAIApiKey!)),
    definition: new OpenAIAssistantDefinition(Secrets.OpenAIModel)
    {
        Instructions = """
            You are an assistant with vision capabilities.
            Extract the readable text from the image.
            """,
        Description = "A chat bot that reads the text in the image.",
        Name = "Specullum",
        Id = "Specullum_01",
        EnableFileSearch = true,
        CodeInterpreterFileIds = [uploadFile.Id]
    });

### ⚡ The chat is invoking Jack Sparrow agent

In [ ]:
var content = await fileService.GetFileContentAsync(uploadFile.Id);

In [ ]:
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

Console.WriteLine($"{AuthorRole.User} > {goal}.");

await foreach (var content in chat.InvokeAsync(agent))
{
    Console.WriteLine($"{content.Role} [{content.AuthorName}] > {content.Content}");
}

### Notice that the agent is fetching additional context from the external file _details.txt_